In [61]:
import numpy as np
import keras
from keras.layers import Dense, Input, LSTM, Dropout
from keras.models import Model, Sequential
from keras.regularizers import l2
from keras.optimizers import Adam



In [62]:
model = Sequential()
model.add(LSTM(20, recurrent_dropout=0.14, recurrent_regularizer=l2(0.01), input_shape=(3,10), return_sequences=False))
model.add(Dropout(0.14))
model.add(Dense(2, activation='softmax'))
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 20)                2480      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 42        
Total params: 2,522
Trainable params: 2,522
Non-trainable params: 0
_________________________________________________________________


In [63]:
def generator(size, form='v'):
    X = np.zeros((size,3,10))
    Y = np.zeros((size,2))
    for batch in xrange(size):
        lu = np.random.randint(1,10) # [1,9]
        mid= np.random.randint(0,lu) # [0,lu)
        ru = np.random.randint(mid+1,10) # [mid+1, 9]
        if form=='v':
            X[batch,0,lu] = 1
            X[batch,1,mid]= 1
            X[batch,2,ru] = 1
            Y[batch,0] = 1
        else:
            X[batch,0,-lu]= 1
            X[batch,1,-mid]=1
            X[batch,2,-ru] =1
            Y[batch,1] = 1
    return X, Y

In [64]:
X, Y = generator(20000, 'v')
tX, tY = generator(20000, 'u')
X = np.append(X,tX,axis=0)
Y = np.append(Y,tY,axis=0)

In [65]:
model.fit(X,Y,epochs=5,shuffle=True)

Epoch 1/5
40000/40000 [==============================] - 21s 523us/step - loss: 0.4648 - acc: 0.7920
Epoch 2/5
40000/40000 [==============================] - 19s 469us/step - loss: 0.2153 - acc: 0.9231
Epoch 3/5
40000/40000 [==============================] - 20s 492us/step - loss: 0.1959 - acc: 0.9248
Epoch 4/5
40000/40000 [==============================] - 19s 470us/step - loss: 0.1876 - acc: 0.9259
Epoch 5/5
40000/40000 [==============================] - 20s 501us/step - loss: 0.1836 - acc: 0.9255


In [70]:
X, Y = generator(1000, 'v')
tX, tY = generator(1000, 'u')
X = np.append(X,tX,axis=0)
Y = np.append(Y,tY,axis=0)
model.test_on_batch(X,Y)

[0.16999197, 0.93400002]

In [71]:
vx, _ = generator(1,'v')
print 'V' if model.predict(vx)[0,0]>0.5 else '^'

_x, _ = generator(1,'u')
print 'V' if model.predict(_x)[0,0]>0.5 else '^'

V
^
